In [8]:
import pandas as pd
import numpy as np

In [9]:
#variáveis de interesse do dataframe original
variaveis_de_interesse = ['id', 'issue_d', 'earliest_cr_line', 'purpose', 'int_rate', 'installment', 'fico_range_low', 'fico_range_high', 'delinq_2yrs', 'pub_rec', 'last_pymnt_d', 'loan_status', 'funded_amnt','annual_inc', 'term']

#lendo o dataframe
df = pd.read_feather("lending_club_clean.feather")
df_small = df[variaveis_de_interesse]

#apenas contratos em dia, quitados ou que entramram em default ou charge off (default com perda)
df_small = df_small[df_small['loan_status'].isin(['Fully Paid','Current', 'Charged Off', 'Default'])]

#renomenado as colunas
df_small = df_small.rename(columns={'issue_d': 'data_celebracao', 
                                    'earliest_cr_line': 'data_primeiro_emprestimo', 
                                    'purpose': 'finalidade_emprestimo', 
                                    'int_rate': 'taxa_juros', 
                                    'installment': 'valor_parcela',   
                                    'delinq_2yrs': 'inadimplencia_2_anos', 
                                    'pub_rec': 'registros_publicos_negativos', 
                                    'last_pymnt_d': 'data_ultimo_pagamento', 
                                    'loan_status': 'status_emprestimo', 
                                    'funded_amnt': 'valor_financiado',
                                    'annual_inc': 'renda_anual',
                                    'term':'parcelas'})
#filtrando os dados
df_small = df_small.dropna()
df_small = df_small[df_small['valor_financiado'] > 0]
df_small = df_small[df_small['renda_anual'] > 0]
df_small = df_small[df_small['valor_parcela'] > 0]
df_small = df_small[df_small['taxa_juros'] > 0]
df_small = df_small[df_small['parcelas'] > 0]
df_small = df_small[df_small['finalidade_emprestimo'].isin(['debt_consolidation', 'credit_card', 'other', 'home_improvement', 'small_business', 'major_purchase', 'educational', 'wedding', 'car', 'medical', 'moving', 'vacation', 'house'])]    

# dicionário de mapeamento
map_finalidade = {
    'debt_consolidation': 'pagamento de dividas',
    'credit_card': 'cartao de credito',
    'other': 'outros',
    'home_improvement': 'reforma residencial',
    'small_business': 'pequeno negocio',
    'major_purchase': 'compra grande valor',
    'educational': 'financiamento estudantil',
    'wedding': 'festa de casamento',
    'car': 'compra de veiculo',
    'medical': 'despesas medicas',
    'moving': 'mudanca',
    'vacation': 'viagem',
    'house': 'compra de imovel'
}

# aplicando o mapeamento
df_small['finalidade_emprestimo'] = df_small['finalidade_emprestimo'].map(map_finalidade)

In [10]:
# Garantindo que as colunas são datetime
df_small['data_celebracao'] = pd.to_datetime(df_small['data_celebracao'])
df_small['data_primeiro_emprestimo'] = pd.to_datetime(df_small['data_primeiro_emprestimo'])

# Calculando o tempo de relacionamento em meses
df_small['tempo_relacionamento'] = (df_small['data_celebracao'].dt.year - df_small['data_primeiro_emprestimo'].dt.year) * 12 + (df_small['data_celebracao'].dt.month - df_small['data_primeiro_emprestimo'].dt.month)

#calculando o escore de crédito como a média entre o fico_range_low e fico_range_high
df_small['escore_fico'] = (df_small['fico_range_high'] + df_small['fico_range_low'])/2
df_small['escore_fico'] = df_small['escore_fico'].apply(lambda x: int(x))

#calculando a renda mensal, a relação parcela/renda e o tempo de relacionamento em meses
df_small['renda_mensal'] = df_small['renda_anual'] / 12

#calculando a relação parcela/renda
df_small['compm_renda'] = df_small['valor_parcela'] / df_small['renda_mensal']

#criando a variável de evento e tempo para análise de sobrevivência
df_small['default'] = df_small['status_emprestimo'].apply(lambda x: 1 if x in ['Charged Off', 'Default'] else 0)

#calculando o tempo de ocorrencia ou não ocorrencia do evento em meses
df_small['T'] = (df_small['data_ultimo_pagamento'].dt.year - df_small['data_celebracao'].dt.year) * 12 + (df_small['data_ultimo_pagamento'].dt.month - df_small['data_celebracao'].dt.month)
df_small['T'] = df_small['T'].apply(lambda x: int(x))

#transfirando as colunas de valor para float e arredondando para 2 casas decimais
df_small['valor_financiado'] = df_small['valor_financiado'].apply(lambda x: round(float(x),2))
df_small['valor_parcela'] = df_small['valor_parcela'].apply(lambda x: round(float(x),2))
df_small['renda_mensal'] = df_small['renda_mensal'].apply(lambda x: round(float(x),2))
#calculando a relação parcela/renda e arredondando para 4 casas decimais
df_small['compm_renda'] = df_small['compm_renda'].apply(lambda x: round(float(x),4))
df_small['taxa_juros'] = df_small['taxa_juros'].apply(lambda x: round(float(x),4))

#deletando as colunas que não serão mais utilizadas
df_small = df_small.drop(columns=['fico_range_low', 'fico_range_high', 'status_emprestimo'])

#outros filtros de dados
df_small = df_small[df_small['escore_fico'] > 0]
df_small = df_small[df_small['tempo_relacionamento'] >= 0]
df_small = df_small[df_small['compm_renda'] >= 0]

#ordenando as colunas
df_small = df_small[['id', 'data_celebracao', 'data_primeiro_emprestimo', 'data_ultimo_pagamento', 'valor_financiado', 'finalidade_emprestimo', 'parcelas', 'taxa_juros', 'valor_parcela', 'tempo_relacionamento', 'escore_fico', 'inadimplencia_2_anos', 'registros_publicos_negativos', 'renda_mensal', 'compm_renda','default','T']]


In [11]:
display(df_small)

,id,data_celebracao,data_primeiro_emprestimo,data_ultimo_pagamento,valor_financiado,finalidade_emprestimo,parcelas,taxa_juros,valor_parcela,tempo_relacionamento,escore_fico,inadimplencia_2_anos,registros_publicos_negativos,renda_mensal,compm_renda,default,T
0,1077501,2011-12-01,1985-01-01,2015-01-01,5000.0,cartao de credito,36,0.1065,162.87,323,737,0,0,2000.00,0.0814,0,37
1,1077430,2011-12-01,1999-04-01,2013-04-01,2500.0,compra de veiculo,60,0.1527,59.83,152,742,0,0,2500.00,0.0239,1,16
2,1077175,2011-12-01,2001-11-01,2014-06-01,2400.0,pequeno negocio,36,0.1596,84.33,121,737,0,0,1021.00,0.0826,0,30
3,1076863,2011-12-01,1996-02-01,2015-01-01,10000.0,outros,36,0.1349,339.31,190,692,0,0,4100.00,0.0828,0,37
4,1075358,2011-12-01,1996-01-01,2017-01-01,3000.0,outros,60,0.1269,67.79,191,697,0,0,6666.67,0.0102,0,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925487,102556443,2017-04-01,1995-04-01,2018-03-01,24000.0,outros,60,0.2399,690.30,264,672,3,2,8916.67,0.0774,1,11
2925488,102653304,2017-04-01,1993-09-01,2019-10-01,10000.0,pagamento de dividas,36,0.0799,313.32,283,727,1,0,5416.67,0.0578,0,30
2925489,102628603,2017-04-01,1993-05-01,2019-05-01,10050.0,pagamento de dividas,36,0.1699,358.26,287,707,0,0,3083.33,0.1162,1,25
2925490,102196576,2017-04-01,1990-05-01,2019-01-01,6000.0,cartao de credito,36,0.1144,197.69,323,672,1,0,3416.67,0.0579,0,21


In [12]:
df_small.to_csv("loan_data_total.csv", index=False, sep=';')

In [13]:
tamanho_gb = df_small.memory_usage(deep=True).sum() / (1024 ** 3)
print(f"Tamanho do dataframe df_small: {tamanho_gb:.2f} GB")

Tamanho do dataframe df_small: 0.39 GB


In [14]:
df_small.head(10000).to_csv("loan_data.csv", index=False, sep=';')